# Import and inspect keras model

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

In [4]:
model = load_model("./models/keras_export")

2022-07-09 14:06:58.366964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-09 14:06:58.368546: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 8, 8)]  0           []                               
                                                                                                  
 input/conv2d (Conv2D)          (None, 192, 8, 8)    193536      ['input_1[0][0]']                
                                                                                                  
 input/bn (BatchNormalization)  (None, 192, 8, 8)    1344        ['input/conv2d[0][0]']           
                                                                                                  
 activation (Activation)        (None, 192, 8, 8)    0           ['input/bn[0][0]']               
                                                                                              

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
# doesn't work: https://github.com/keras-team/keras/issues/11746
embedding = keras.Model(inputs=model.input, outputs=model.get_layer("activation_45"), name="embedding_model")

In [ ]:
# workaround: sequential api
sequential = keras.Sequential()
sequential.add(model.get_layer("input_1"))
sequential.add(model.get_layer("activation_45"))
sequential.compile()

In [ ]:
input = tf.random.uniform((1,112,8,8))
sequential.predict(input)

# Export keras model to onnx

In [7]:
import tf2onnx

In [8]:
spec = (tf.TensorSpec((None, 112, 8, 8), tf.float32, name="input"),)
output_path = model.name + ".onnx"
tf2onnx.convert.from_keras(model=model, opset=12, input_signature=spec, output_path=output_path)

2022-07-09 14:20:37.065019: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 
2022-07-09 14:20:37.065193: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-07-09 14:20:38.599208: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-07-09 14:20:38.940387: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 
2022-07-09 14:20:38.940614: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


# Test onnx model

In [ ]:
# https://github.com/microsoft/onnxruntime/issues/3229
import numpy as np
import onnxruntime
sess = onnxruntime.InferenceSession(output_path)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
scores = sess.run([output_name], {input_name: np.random.random((1,112,8,8)).astype(np.float32)})[0]
scores, output_name

NameError: name 'output_path' is not defined

# Export onnx model to tensorflow

In [ ]:
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("model.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("tf/model")

# Export onnx model to pytorch

import torch
from onnx2torch.converter import convert

# Path to ONNX model
onnx_model_path = 'model.onnx'

# Or you can load a regular onnx model and pass it to the converter
onnx_model = onnx.load(onnx_model_path)
torch_model_2 = convert(onnx_model)

In [ ]:
torch_model_2.forward(torch.rand(112,8,8))

In [ ]:
torch.save(torch_model_2, "pytorch/model.pt")